In [1]:
import sympy as sp
from sympy import init_printing
init_printing()
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent))
from cores.dynamical_systems.strict_feedback_3d import StrictFeedback3D
import torch
import numpy as np

In [2]:
a1 = -1.1
a2 = 1.2
b1 = -1.3
b2 = 1.4
c1 = -1.5
c2 = 1.6


In [3]:
system_torch = StrictFeedback3D(a1=a1, a2=a2, b1=b1, b2=b2, c1=c1, c2=c2, dtype=torch.float32)

In [4]:
x1, x2, x3 = sp.symbols(r'x_1 x_2 x_3')
u = sp.symbols(r'u')
states = sp.Matrix([x1, x2, x3])
controls = sp.Matrix([u])

In [5]:
f1 = a1*x1 + a2*x2
f2 = b1*x2 + b2*x3
f3 = c1*x3**2 + c2*u

f = sp.Matrix([f1, f2, f3])

# Test f(x,u)

In [6]:
f_func = sp.lambdify((states, controls), f)
state_np = np.array([1., 2., 3.0], dtype=np.float32)
control_np = np.array([3.], dtype=np.float32)
print(f_func(state_np, control_np))


[[ 1.3000001]
 [ 1.5999999]
 [-8.7      ]]


In [7]:
state_torch = torch.tensor(state_np, dtype=torch.float32).unsqueeze(0)
control_torch = torch.tensor(control_np, dtype=torch.float32).unsqueeze(0)
f_torch = system_torch(state_torch, control_torch)
print(f_torch)

tensor([[ 1.3000,  1.6000, -8.7000]])


# Test df/dx

In [8]:
df_dx = f.jacobian(states)
df_dx_func = sp.lambdify((states, controls), df_dx)
print(df_dx_func(state_np, control_np))

[[-1.1  1.2  0. ]
 [ 0.  -1.3  1.4]
 [ 0.   0.  -9. ]]


In [9]:
df_dx_torch = system_torch.f_dx(state_torch, control_torch)
print(df_dx_torch)

tensor([[[-1.1000,  1.2000,  0.0000],
         [ 0.0000, -1.3000,  1.4000],
         [ 0.0000,  0.0000, -9.0000]]])


# Test df/du

In [10]:
df_du = f.jacobian(controls)
df_du_func = sp.lambdify((states, controls), df_du)
print(df_du_func(state_np, control_np))

[[0. ]
 [0. ]
 [1.6]]


In [11]:
df_du_torch = system_torch.f_du(state_torch, control_torch)
print(df_du_torch)

tensor([[[0.0000],
         [0.0000],
         [1.6000]]])


# d^2f/dx^2

In [12]:
f1_dxdx = f1.diff(states).jacobian(states)
print(f1_dxdx)

Matrix([[0, 0, 0], [0, 0, 0], [0, 0, 0]])


In [13]:
f2_dxdx = f2.diff(states).jacobian(states)
print(f2_dxdx)

Matrix([[0, 0, 0], [0, 0, 0], [0, 0, 0]])


In [15]:
f3_dxdx = f3.diff(states).jacobian(states)
print(f3_dxdx)

Matrix([[0, 0, 0], [0, 0, 0], [0, 0, -3.00000000000000]])
